In [1]:
import anndata as ad
import scvi
import scanpy as sc
import mrvi
import pandas as pd
import scipy as sp
import numpy as np

Global seed set to 0
/home/dennis/miniconda3/envs/compbioGPU/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/dennis/miniconda3/envs/compbioGPU/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/dennis/miniconda3/envs/compbioGPU/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


# Exploring AnnData objects
(don't delete this, keep for future reference)

In [2]:
# # Listing vars, i.e. cell barcodes (auto ignores the first column titled 'gene', unlike pandas)
# adata.var_names

# # Getting the gene names
# adata.obs_names

# # Getting the counts
# adata.X

# # Sparsifying if raw and de-sparsifying if not
# adata.X = sp.sparse.csr_matrix(adata.X)
# adata.X.toarray()

In [3]:
untreated_1_data = "./../untreated1.h5ad"
adata = sc.read(untreated_1_data)

In [ ]:
# TODO: figure out how to use this pipeline with multisample data, then read in all the samples, transpose 'em, combine into a single datastructure, pre-process, and go.  

In [4]:
# From gene x cell to cell x gene matrix (as downstream functions expect)
adata = adata.transpose()

In [5]:
# Confirming the data is as expected
# adata.obs
# adata.var_names

In [11]:
# Trying to coerce the adata to work!
adata.obs["sample"] = np.random.choice(15, size=adata.shape[0])

In [23]:
adata.obs["sample"]

410143

In [24]:
# testing some weirdness
batch_si = len(adata.obs)
n_batches = 1
batch = []
for i in range(n_batches):
    batch += [f"batch_{i}"] * batch_si

adata.obs["batch"] = pd.Categorical(batch)
# type(pd.Categorical(batch))
# batch

In [25]:
# Sample (e.g. donors, perturbations, etc.) should go in sample_key
# Sites, plates, and other factors should go in categorical_nuisance_keys
# mrvi.MrVI.setup_anndata(adata)
mrvi.MrVI.setup_anndata(adata, sample_key="sample", categorical_nuisance_keys=["batch"])

In [26]:
for linear_decoder_uz in [True, False]:
    for linear_decoder_zx in [True, False]:
        model = mrvi.MrVI(
            adata,
            n_latent_sample=5,
            linear_decoder_zx=linear_decoder_zx,
            linear_decoder_uz=linear_decoder_uz,
        )
        model.train(1, check_val_every_n_epoch=1, train_size=0.5)
        model.history

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/1: 100%|██████████| 1/1 [00:29<00:00, 29.83s/it, loss=149, v_num=1]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Epoch 1/1: 100%|██████████| 1/1 [00:30<00:00, 30.36s/it, loss=38.8, v_num=1]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Epoch 1/1: 100%|██████████| 1/1 [00:35<00:00, 35.33s/it, loss=50.4, v_num=1]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Epoch 1/1: 100%|██████████| 1/1 [00:31<00:00, 31.97s/it, loss=35.8, v_num=1]


In [27]:
model = mrvi.MrVI(
        adata,
        n_latent_sample=5,
        linear_decoder_zx=True,
        linear_decoder_uz=True,
        linear_decoder_uz_scaler=True,)
model.train(1, check_val_every_n_epoch=1, train_size=0.5)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/1: 100%|██████████| 1/1 [00:28<00:00, 28.82s/it, loss=46.3, v_num=1]


In [28]:
model.get_latent_representation()

100%|██████████| 3205/3205 [03:14<00:00, 16.49it/s]


array([[ -1.727188  ,  -0.96002334,  -0.15920436, ...,  -0.10573398,
         -1.0412022 ,   5.924918  ],
       [ -0.87542987, -11.801965  ,  -7.824737  , ...,   4.688609  ,
          2.318755  ,  -7.272049  ],
       [ -3.5731335 ,   1.2332203 ,   1.059418  , ...,   2.8871663 ,
          4.6618104 ,   4.4962626 ],
       ...,
       [ -6.031334  ,  -8.389166  , -13.810276  , ...,  23.443542  ,
         15.874462  , -25.70056   ],
       [ -0.79780513,  -1.0290052 ,   1.5093315 , ...,  -1.4778361 ,
         -6.384366  , -10.192635  ],
       [ -7.3813    ,  -7.558058  , -13.311421  , ...,  14.935344  ,
          1.5988284 ,  -4.638628  ]], dtype=float32)

In [30]:
assert model.get_local_sample_representation().shape == (adata.shape[0], 15, 10)
assert model.get_local_sample_representation(return_distances=True).shape == (
    adata.shape[0],
    15,
    15,
)
# tests __repr__
print(model)

100%|██████████| 1603/1603 [00:56<00:00, 28.26it/s]


Training status: Trained

In [29]:
# mrvi_model.train()

In [31]:
# # Get z representation
adata.obsm["X_mrvi_z"] = model.get_latent_representation(give_z=True)
# # Get u representation
adata.obsm["X_mrvi_u"] = model.get_latent_representation(give_z=False)
# # Cells by n_sample by n_latent
cell_sample_representations = model.get_local_sample_representation()
# # Cells by n_sample by n_sample
cell_sample_sample_distances = model.get_local_sample_representation(return_distances=True)

100%|██████████| 1603/1603 [00:47<00:00, 33.42it/s]
